# Transpiling Chemistry circuits with Qiskit and Pytket

### Chemistry

In [ ]:
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from sympy import Symbol
import warnings
warnings.filterwarnings(action='ignore')

First we need a quantum circuit to work with. Chemistry and solid state physics are proving to be the first standout application of NISQ quantum devices. We are able to simulate the quantum state of small molecules using a quantum device. As devices improve, we may be able to accurately calculate properties of molecules that classical computers cannot. 

We don't need to worry about the specifics of what's going on here, broadly we are setting up a Lithium Hydride molecule to be simulated and we are going to get a circuit from Qiskit Aqua that prepares a trial state on the quantum device. We will then compile this using pytket.

In [ ]:
molecule_str = 'Li .0 .0 .0; H .0 .0 1.6'
basis = 'sto3g'
qubit_mapping = 'jordan_wigner'


In [ ]:
driver = PySCFDriver(molecule_str, unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis=basis)
molecule = driver.run()

map_type = 'jordan_wigner'

h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2

ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp.chop(10**-10)

# setup HartreeFock state
HF_state = HartreeFock(qubitOp.num_qubits, num_spin_orbitals, num_particles, map_type, 
                       False)

# setup UCCSD variational form
var_form = UCCSD(qubitOp.num_qubits, depth=1, 
                   num_orbitals=num_spin_orbitals, num_particles=num_particles, 
                   active_occupied=[0], active_unoccupied=[0, 1],
                   initial_state=HF_state, qubit_mapping=map_type, 
                   two_qubit_reduction=False, num_time_slices=1)

number_amplitudes = len(var_form._single_excitations) + len(var_form._double_excitations)
amplitudes = [1e-4]*number_amplitudes
circuit = var_form.construct_circuit(amplitudes)
circuit.size()

The circuit starts off with 756 quantum gates. This circuit is certainly too large to be feasibly run on today's quantum hardware, but by no means out of reach for near term NISQ devices. This is also small enough to easily be simulated.

### Transpilation

To reduce the resource requirements we can try "transpiling" the circuit. Transpiling is graph rewriting performed to optimise the circuit, leaving it in the same data structure as before but with fewer resource requirements. This rewriting can also change the gate set and satisfy the constraints of a connectivity graph. Now, let's try optimising this circuit using the highest level of Qiskit's native transpiler.

In [ ]:
from qiskit.compiler import transpile

In [ ]:
circuit_after = transpile(circuit,basis_gates=['u3','cx'],coupling_map=None, optimization_level=3)
circuit_after.size()

We have reduced the gate count by 221! A respectable number.

In [ ]:
print(circuit_after)

Now, let's convert the original circuit to a pytket circuit:

In [ ]:
from pytket.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket import Circuit, OpType

In [ ]:
tkcirc = qiskit_to_tk(circuit)
print(tkcirc.n_gates)
print(tkcirc.n_gates_of_type(OpType.CX))

You can see that the circuit in pytket format has the same number of gates as the original circuit in Qiskit format. We can also see now that the circuit has 424 CX gates in it. These two qubit gates are entangling gates, and typically have much greater error rates associated with them on all kinds of physical hardware -- around an order of magnitude worse.

Now, let's import some pytket classes. We will need the `Circuit` class, as well as the `CompilationUnit` class. This holds a `Circuit`, and also holds some information to track the compilation of the `Circuit`.

We will also need the `SynthesiseIBMPass` and `PauliSimpPass` -- these are two "StandardPasses" which are the building blocks of the pytket transpilation process. Lastly, we need the `SequencePass`. This is a *combinator*, which combines other passes in ways ranging from the simple (sequential composition like this one) to the complex (repeat until a custom user-defined function on the `Circuit` returns true).

In [ ]:
from pytket import CompilationUnit, SynthesiseIBMPass, PauliSimpPass, SequencePass

In [ ]:
print(SynthesiseIBMPass)

Pytket passes are made of preconditions and postconditions, which give you assertions about what requirements the `Circuit` must have for this pass to successfully run, as well as guarantees about which properties `Circuit` will have associated with it afterwards. This should be familiar to anyone who has dealt with Hoare Logic, or contracts in software development.

So we can see that the `SynthesiseIBMPass` requires that the input `Circuit` has no gates with classical control on them -- this would get in the way of the rewrite rules, and prevent optimisation. It guarantees that the `Circuit` afterwards will have gates only from the gateset shown above. It also guarantees that there are no gates which act on more than two qubits at once -- although this is obvious from the previous guarantee.

In [ ]:
cu1 = CompilationUnit(tkcirc)
print(SynthesiseIBMPass.apply(cu1))
tkcirc_after_1 = cu1.get_circ()
print(tkcirc_after_1.n_gates)
print(tkcirc_after_1.n_gates_of_type(OpType.CX))

A Pytket pass returns a `bool`, which is `True` if the pass modified the circuit at any point in its graph rewriting, and `False` otherwise. We can see that our `SynthesiseIBMPass` has reduced the total number of gates to 482. The CX count, however, has been reduced by only 88, so the errors on this circuit will not be much different from before.

We can print the `CompilationUnit` out, and we see it has a cache for tracking some information about the circuit to ensure composition of passes is working correctly.

In [ ]:
print(cu1)

Now let's instead try a pass which was designed specifically for chemistry circuits: the `PauliSimpPass`. See https://arxiv.org/pdf/1906.01734.pdf for a description of how this works -- it is derived using a graphical calculus taken from the field of categorical quantum mechanics :)

In [ ]:
cu2 = CompilationUnit(tkcirc)
print(PauliSimpPass.apply(cu2))
tkcirc_after_2 = cu2.get_circ()
print(tkcirc_after_2.n_gates)
print(tkcirc_after_2.n_gates_of_type(OpType.CX))

This pass took a bit longer, and has reduced the total gate count a bit more, to 418, although the CX count difference is big this time! We have reduced it by 226, to less than 47% of the original CX count.

Now, let's compose these passes and see what we get out. `SequencePass` automatically checks the preconditions and postconditions of the composite passes and verifies that the composition is valid.

In [ ]:
combo_pass = SequencePass([PauliSimpPass,SynthesiseIBMPass])
print(combo_pass)
cu3 = CompilationUnit(tkcirc)
print(combo_pass.apply(cu3))
tkcirc_after_3 = cu3.get_circ()
print(tkcirc_after_3.n_gates)
print(tkcirc_after_3.n_gates_of_type(OpType.CX))

We now have only 353 total gates, and only 198 CX gates. This is still not plausibly runnable on real quantum computers, but it is much closer.

In [ ]:
circuit = tkcirc_after_3
print(tk_to_qiskit(circuit))

### Backends

In [ ]:
from pytket.backends import AerStateBackend
backend = AerStateBackend()
print(backend.valid_circuit(circuit))

In [ ]:
for pred in backend.required_predicates:
    print(pred)

If the circuit does not satisfy the constraints of the backend, pytket can automatically run a pass which will constrain it to do so:

In [ ]:
if not backend.valid_circuit(circuit):
    circuit = backend.compile_circuit(circuit)
assert(backend.valid_circuit(circuit))

In [ ]:
results = backend.get_state(circuit)
print(len(results))

We can output to several different backends. For this demo we will use simulators -- from Qiskit we offer the Aer Statevector generator, as well as the Aer QASM Simulator. We also offer the ProjectQ simulator. We offer backends to quantum devices too: the IBMQ devices are accessible via pytket, and we also have a backend to the Honeywell ion trap machine. This will be made available in a future release.

Let's use the ProjectQ backend. This time, our circuit will not be in the correct gateset. We can let pytket take care of that.

In [ ]:
from pytket.backends.projectq import ProjectQBackend
# from pytket.projectq import ProjectQBackend
projectq_backend = ProjectQBackend()
print(projectq_backend.valid_circuit(circuit))

In [ ]:
if not projectq_backend.valid_circuit(circuit):
    circuit = projectq_backend.compile_circuit(circuit)
assert(projectq_backend.valid_circuit(circuit))

In [ ]:
results2 = projectq_backend.get_state(circuit)

 ### Routing - you need an account to do this section. I will not run it now!

In [ ]:
from pytket.backends.ibm import IBMQBackend

In [ ]:
melbourne = IBMQBackend("ibmq_16_melbourne")
print(melbourne.required_predicates)

In [ ]:
if not melbourne.valid_circuit(circuit):
    circuit = melbourne.compile_circuit(circuit)
assert(melbourne.valid_circuit(circuit))

Not only has the circuit been converted to the correct gate set, it has also been routed to satisfy the connectivity graph. Routing is (at least) an NP-complete problem in general, so heuristics are necessary. We use dynamic routing from our paper: https://arxiv.org/pdf/1902.08091.pdf. We have benchmarks for all of these, and an updated version of our software is being released this month.